# Matrix Assembly

In [1]:
using LinearAlgebra, SparseArrays
using BenchmarkTools

In [2]:
function generate_mesh(Nel)
    x = 0:1/Nel:1;

    # Collect points (left and right nodes for each element) and edges (connectivity: indices of left and right nodes)
    N = length(x);
    points = collect( [x[i], x[i+1]] for i in 1:N-1) 
    edges  = collect( [i, i+1] for i in 1:N-1); 

    elem_ids = 1:length(edges)
    e_group = ones(size(edges));
    
    #..Set the source function 
    fsource(group_id) = 1;
    fsource_elem = map(fsource, e_group);


    return N, points, edges, x, elem_ids, e_group, fsource_elem
end

generate_mesh (generic function with 1 method)

## Triple Loop

In [3]:
function assemble_matrices(edges, points, N, fsource_elem)
    #..Initialize global matrix and right-hand side value 
    A = zeros(Float64, N, N);
    f = zeros(Float64, N); 

    #..Perform loop over elements and assemble global matrix and vector 
    for (i, nodes) = enumerate(edges)
        xl, xr = points[i,:][1]
        h = xr - xl;

        floc = fsource_elem[i] * h / 2 * [1; 1];
        Aloc = 1 / h * [1 -1; -1 1];

        #....perform loop over nodes of the current element
        #....and add local contribution Aloc to global matrix entity A
        for j = 1:2
            I = nodes[j];
            f[I] += floc[j]
            for k = 1:2
                J = nodes[k];
                A[I,J] += Aloc[j,k]
            end
        end

    end

    #..handle the boundary conditions in the matrix and right-hand side vector 
    A[1,:]   = zeros(N);  A[1,1]     = 1;      f[1]   = 0;
    A[end,:] = zeros(N);  A[end,end] = 1;      f[end] = 0;
    
    return A, f
end

assemble_matrices (generic function with 1 method)

In [4]:
N, points, edges, _, _, _, fsource_elem = generate_mesh(10000);

@benchmark A, f = assemble_matrices(edges, points, N, fsource_elem)

BenchmarkTools.Trial: 15 samples with 1 evaluation.
 Range (min … max):  138.615 ms …    1.099 s  ┊ GC (min … max):  0.11% …  9.37%
 Time  (median):     274.242 ms               ┊ GC (median):    22.36%
 Time  (mean ± σ):   299.739 ms ± 228.828 ms  ┊ GC (mean ± σ):  19.96% ± 15.94%

  ▃    ▃  ▃█ ▃                                                   
  █▇▁▁▇█▇▁██▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  139 ms           Histogram: frequency by time           1.1 s <

 Memory estimate: 767.75 MiB, allocs estimate: 60009.

## Single Loop

In [5]:
function assemble_matrices(edges, points, N, fsource_elem)    
    #..Initialize global matrix and right-hand side value 
    A = zeros(Float64, N, N);
    f = zeros(Float64, N); 

    #..Perform loop over elements and assemble global matrix and vector 
    for (i, nodes) = enumerate(edges)
        xl, xr = points[i,:][1]
        h = xr - xl;

        floc = fsource_elem[i] * h / 2 * [1; 1];
        Aloc = 1 / h * [1 -1; -1 1];

        f[nodes] += floc;
        A[nodes, nodes] += Aloc;
    end

    #..handle the boundary conditions in the matrix and right-hand side vector 
    A[1,:]   = zeros(N);  A[1,1]     = 1;      f[1]   = 0;
    A[end,:] = zeros(N);  A[end,end] = 1;      f[end] = 0;
    
    return A, f
end

assemble_matrices (generic function with 1 method)

In [6]:
N, points, edges, _, _, _, fsource_elem = generate_mesh(10000);

@benchmark A = assemble_matrices(edges, points, N, fsource_elem)

BenchmarkTools.Trial: 17 samples with 1 evaluation.
 Range (min … max):  160.367 ms … 342.028 ms  ┊ GC (min … max):  0.32% … 33.44%
 Time  (median):     270.010 ms               ┊ GC (median):    21.31%
 Time  (mean ± σ):   269.316 ms ±  44.885 ms  ┊ GC (mean ± σ):  23.41% ± 10.07%

  ▁                ▁        ▁ ▁▁  ▁▁▁ ██        ▁     ▁▁   ▁  ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁██▁▁███▁██▁▁▁▁▁▁▁▁█▁▁▁▁▁██▁▁▁█▁▁█ ▁
  160 ms           Histogram: frequency by time          342 ms <

 Memory estimate: 771.10 MiB, allocs estimate: 100009.

## Single Loop with Pre-allocation

## Single Loop with Sparse

In [7]:
function assemble_matrices(edges, points, N, fsource_elem)
    # Initialize vectors for sparse matrix construction
    I = zeros(Int64, 4*length(edges))
    J = similar(I);
    V = zeros(Float64, length(I));
    
    # Initialize right-hand side vector
    f = zeros(Float64, N); 
    
    #..Perform loop over elements and assemble global matrix and vector 
    for (i, nodes) = enumerate(edges)
        idx = 4*(i-1) + 1 : 4*i;
        h   = points[i][2] - points[i][1];
        
        #
        f[nodes] += fsource_elem[i] * h / 2 * [1; 1];
        
        # Matrix contribution
        I[idx] = nodes[[1, 2, 1, 2]];
        J[idx] = nodes[[1, 1, 2, 2]];
        V[idx] = 1/h * [1 -1 -1 1];
    end
    
    A = sparse(I, J, V)
    
    #..handle the boundary conditions in the matrix and right-hand side vector 
    A[1,:]   = zeros(N);  A[1,1]     = 1;      f[1]   = 0;
    A[end,:] = zeros(N);  A[end,end] = 1;      f[end] = 0;
    
    return A, f
end

assemble_matrices (generic function with 1 method)

In [8]:
N, points, edges, _, _, _, fsource_elem = generate_mesh(10000);

@benchmark A, f = assemble_matrices(edges, points, N, fsource_elem)

BenchmarkTools.Trial: 865 samples with 1 evaluation.
 Range (min … max):  4.058 ms … 24.755 ms  ┊ GC (min … max):  0.00% … 78.61%
 Time  (median):     5.272 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.774 ms ±  1.858 ms  ┊ GC (mean ± σ):  14.77% ± 17.11%

  ██▄                                                         
  ████▅▃▄▄█▇▆█▅▅▅▃▃▂▃▅▄▆▆▅▄▃▃▃▃▃▃▃▃▄▃▃▃▃▂▃▂▂▃▃▂▂▁▂▂▁▂▂▂▁▂▁▁▂ ▃
  4.06 ms        Histogram: frequency by time        11.6 ms <

 Memory estimate: 12.21 MiB, allocs estimate: 100061.

## No Loop
This does not seem to be a very efficient use of ``mapreduce``, next step: study the use of ``map()`` and ``mapreduce()`` and maybe write about this in a notebook about general Julia usage?

Note: still missing assembly of $f$ vector

In [9]:
function compute_element_area(elem_id, e, p)
    area_id = p[e[elem_id][2]] - p[e[elem_id][1]]
    return area_id
end

compute_element_area (generic function with 1 method)

In [10]:
function assemble_matrices(elem_ids, edges, x, N)
    Atempl = [1, -1, -1, 1];
    
    # Calculate element length h
    elem_area = map(elem_id -> compute_element_area(elem_id, edges, x), elem_ids);
    
    # Generate index vectors
    I = mapreduce(e -> e[[1, 2, 1, 2]], vcat, edges)
    J = mapreduce(e -> e[[1, 1, 2, 2]], vcat, edges)
    
    # Generate matrix contributions
    V = mapreduce(h -> Atempl / h, vcat, elem_area);
    
    return sparse(I, J, V, N, N)
end

assemble_matrices (generic function with 1 method)

In [11]:
N, points, edges, x, elem_ids = generate_mesh(10000);
@benchmark A = assemble_matrices(elem_ids, edges, x, length(x))

BenchmarkTools.Trial: 92 samples with 1 evaluation.
 Range (min … max):  42.413 ms … 212.333 ms  ┊ GC (min … max): 17.89% … 74.96%
 Time  (median):     50.816 ms               ┊ GC (median):    18.37%
 Time  (mean ± σ):   54.790 ms ±  18.693 ms  ┊ GC (mean ± σ):  20.93% ±  6.12%

   ▃█ ▁▃ █   ▁▄ ▁       ▁   ▃                                   
  ▆██▄██▆█▆▆▆██▄█▆▄▄▆▁▁▁█▇▁▆█▆▁▄▄▁▄▆▁▆▁▁▄▄▁▄▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  42.4 ms         Histogram: frequency by time         83.9 ms <

 Memory estimate: 299.78 MiB, allocs estimate: 85488.